# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — проверьте четыре гипотезы:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок? 
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

# Оглавление: <a id='intro'></a>  
- [Знакомство с данными](#1)  
- [Обработка пропусков](#2)  
- [Замена типа данных](#3)  
- [Обработка дубликатов](#4)  
- [Лемматизация](#5)  
- [Категоризация данных](#6)  
- [Проверка гипотез](#7)  
- [Общий вывод](#8) 

<a id='1'></a> 

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [2]:
import pandas as pd

data = pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info() # получим общую информацию о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.describe() # усилим анализ

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Итак в таблице есть двенадцать столбцов. Тип данных в столбцах разный — `object`, `int64`, `float64`.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

**Вывод**

В каждой строке таблицы — данные о клиенте. Часть колонок описывает самого клиента: возраст, пол, оразование, семейное положение, количество детей. Остальные данные рассказывают о пользователе с точки зрения его финансовых возможностей: трудовой стаж, тип занятости, имел ли задолжности по возврату кредитов, ежемесячный дход, цель получения кредита.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных и артефакты — значения, которые не отражают действительность. Например, отрицательное количество дней трудового стажа.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

[Назад к оглавлению](#intro)

<a id='2'></a> 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
data.isna().sum() # поиск и подсчет пропущеных значений во всей таблице

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Мы обнаружили пропуски в столбцах: `days_employed` и `total_income`. Появление пропусков в стаже работы возможно из за ошибок при занесении данных о клиенте, а отсутствие информации о доходе может быть связано с тем, что клиент на данный момент никак не подтвердил свой доход. Также в стаже работы есть отрицательные значения и есть выбросы в виде стажа превышающего человеческую жизнь, это тоже необходимо исправить. Скорей всего это связано с тем, что эти значения представленны не в днях, а в часах (возможно данные поступали из разных источников).

In [6]:
data['days_employed'] = data['days_employed'].apply(abs) # исправим значения на положительные
data['education'] = data['education'].str.lower() # установим единый формат текста в столбце education

Согласно данным из [Wikipedia][1] продолжительность жизни в России в среднем составляет около 77 лет, а работать можно с 16 лет. Но мы в принципе можем просто взять максимальный возраст клиентов из данных, вычесть 16 и умножить на 365, чтобы узнать максимально возможный трудовой стаж в днях для нашего случая.






[1]: https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%BE%D0%B4%D0%BE%D0%BB%D0%B6%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D1%8C_%D0%B6%D0%B8%D0%B7%D0%BD%D0%B8_%D0%B2_%D1%81%D1%83%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D0%B0%D1%85_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B9%D1%81%D0%BA%D0%BE%D0%B9_%D0%A4%D0%B5%D0%B4%D0%B5%D1%80%D0%B0%D1%86%D0%B8%D0%B8#:~:text=%D0%9E%D0%B6%D0%B8%D0%B4%D0%B0%D0%B5%D0%BC%D0%B0%D1%8F%20%D0%BF%D1%80%D0%BE%D0%B4%D0%BE%D0%BB%D0%B6%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D1%8C%20%D0%B6%D0%B8%D0%B7%D0%BD%D0%B8%20%D0%B2%20%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8%20%D0%B2%20%D1%81%D1%80%D0%B5%D0%B4%D0%BD%D0%B5%D0%BC%20%D1%81%D0%BE%D1%81%D1%82%D0%B0%D0%B2%D0%BB%D1%8F%D0%B5%D1%82%20%D0%BE%D0%BA%D0%BE%D0%BB%D0%BE%2077,58%20(%D0%BF%D0%BE%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D0%BC%20%D0%BD%D0%B0%202018

In [7]:
print(data['dob_years'].max()) # максимальный возраст
print((data['dob_years'].max() - 16) * 365) # максимальный трудовой стаж в днях
# поскольку сейчас тип данных float, то возьмем значение немного больше

75
21535


In [8]:
data.loc[data['days_employed'] > 22000, 'days_employed'] = data['days_employed'] / 24 # переводим значения стажа из часов в дни

Кроме исправления отрицательных значений трудового стажа, мы перевели значения сохраненные в часах в дни, как то описывает документация. Также привели шрифт к единому формату в столбце `education`.

Для того чтобы корректно заполнить пропуски в данных стоит проверить нет ли похожих ошибок в других столбцах, например в количестве детей.

In [9]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Как видно есть отрицательное значение, очевидно что такого не может быть. Но важно понять на что исправить это значение на `1` или `0`. Посмотрим на возраст и семейное положение этих клиентов.

In [10]:
display(data[data['children'] == -1][['dob_years', 'family_status']])

,dob_years,family_status
291,46,гражданский брак
705,50,женат / замужем
742,57,женат / замужем
800,54,Не женат / не замужем
941,57,женат / замужем
1363,55,женат / замужем
1929,38,Не женат / не замужем
2073,42,в разводе
3814,26,гражданский брак
4201,41,женат / замужем


Судя по возрасту и семейному статусу изначальное значение `-1`, вероятнее всего подразумевает наличие одного ребенка. Также мы заметили возможность изменить формат шрифта в столбце `family_status`. Проверим результат.

In [11]:
data['children'] = data['children'].apply(abs)
data['family_status'] = data['family_status'].str.lower()
data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Теперь проверим что на счет тех у кого `20` детей, кажется что здесь тоже ошибка в данных.

In [12]:
data[data['children'] == 20][['dob_years', 'family_status']].head(10)

,dob_years,family_status
606,21,женат / замужем
720,44,женат / замужем
1074,56,женат / замужем
2510,59,вдовец / вдова
2941,0,женат / замужем
3302,35,не женат / не замужем
3396,56,женат / замужем
3671,23,не женат / не замужем
3697,40,гражданский брак
3735,26,не женат / не замужем


In [13]:
data['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Действительно, выявились новые ошибки в данных:
* **Во-первых**, наличие `20` детей маловероятно, так как, людей с большим количеством детей все меньше и меньше. А тут, с двадцатью их больше, чем тех у кого всего пять. Для ответа на первую гипотезу нам важен факт наличия детей у клиента, поэтому мы не будем удалять эти данные, но основываясь на сказанном выше, мы изменим это значение на `2`, так как на это указывает и семейное положение в большинстве случаев.
* **Во-вторых**, есть клиенты с нулевым возрастом. Для них можно взять медианное значение с учетом семейного положения и количества детей.

In [14]:
data.loc[data['children'] == 20, 'children'] = 2
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Заполним нулевые значения в возрасте клиентов.

In [15]:
data_dob_years = data.groupby(['children', 'family_status']).agg({'dob_years': 'median'}).rename(columns={'dob_years': 'median_dob_years'})
data_dob_years['median_dob_years'] = data_dob_years['median_dob_years'].astype('int') # меняем тип данных, т.к. в data возраст имеет тип int
data = data.merge(data_dob_years, on=['children', 'family_status']) # объединяем таблицы по соответствующим столбцам
data.loc[data['dob_years'] == 0, 'dob_years'] = data.loc[data['dob_years'] == 0, 'median_dob_years'] # заполняем нули медианным возрастом
data.drop('median_dob_years', axis=1, inplace=True) # удаляем столбец из data
data_dob_years = pd.DataFrame(None) # чистим фрейм, чтобы не занимал память, так как он нам теперь не нужен

Проверим, остались ли клиенты с нулевым возрастом.

In [16]:
data['dob_years'][data['dob_years'] < 20].unique() # клиентов с нулевым возрастом больше нет

array([19])

Интересно узнать, а нет ли ошибок или пропусков в значениях столбца `gender`.

In [17]:
data['gender'].value_counts() # какие значения представлены в gender

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

`XNA` это пропуск в данных о поле клиента. Мы можем изменить значение на женский, так как женщин в выборке больше и наше изменение ни на что не повлияет.

In [18]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'
data['gender'].value_counts() # проверим результат

F    14237
M     7288
Name: gender, dtype: int64

In [19]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,1,972.364419,26,среднее,1,женат / замужем,0,F,сотрудник,0,116820.904450,строительство собственной недвижимости
3,1,1311.604166,36,среднее,1,женат / замужем,0,M,сотрудник,0,209552.644578,недвижимость
4,1,717.274324,26,высшее,0,женат / замужем,0,F,сотрудник,0,187863.237306,строительство собственной недвижимости


Теперь перейдем к заполнению пропусков в трудовом стаже, факторы которые могут влиять на трудовой стаж это:
* `education` - образование, клиенты получившие высшее образование, скорей всего имеют меньший трудовой стаж, но нужно учесть еще и возраст;
* `dob_years` - возраст клиентов;
* `gender` - пол, так как на стаж могут повлиять возможные декреты и разный пенсионный возраст;
* `income_type` - тип занятости, в нашем случае представленных професий не так много, но есть профессии которые выходят на пенсию раньше.

In [20]:
data_days_employed = data.groupby(['dob_years', 'gender']).agg({'days_employed': 'median'}).rename(columns={'days_employed': 'median_days_employed'})
data_days_employed[data_days_employed['median_days_employed'].isna()]

,,median_days_employed
dob_years,gender,


<div class="alert alert-info" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid">

Не удается заполнить все пропуски в трудовом стаже медианой, возможно это связано с тем, что для некоторых критериев не удается найти медианное значение.
Поэтому пока остановимся на двух критриях (при выполнении которых пропусков не остается): возраст и пол.</div>

In [21]:
data_days_employed = data.groupby(['dob_years', 'gender']).agg({'days_employed': 'median'}).rename(columns={'days_employed': 'median_days_employed'})
data = data.merge(data_days_employed, on=['dob_years', 'gender'])
data.loc[data['days_employed'].isna(), 'days_employed'] = data.loc[data['days_employed'].isna(), 'median_days_employed']
data.drop('median_days_employed', axis=1, inplace=True) # удаляем столбец из data
data_days_employed = pd.DataFrame(None) # чистим фрейм, чтобы не занимал память, так как он нам теперь не нужен

In [22]:
data.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Осталось избавиться от пропусков в столбце ежемесячных доходов, для этого так же возьмем медиану с учетом типа занятости клиентов.

In [23]:
for inctype in data['income_type'].unique(): # цикл пройдет по всем уникальным значениям типа занятости
    try:
        median = data.loc[data['income_type'] == inctype, 'total_income'].median() # определим медиану для каждого типа занятости
        data.loc[(data['total_income'].isna()) & (data['income_type'] == inctype), 'total_income'] = median # заполним пропуски медианой с учетом типа занятости клиента
    except:
        print('Упс, что-то пошло не так :)')

print(data.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Вывод**

Мы исправили все пропущенные значения, даже те которые не были видны на первый взгляд. Можно переходить к следующему этапу предобработки данных.

[Назад к оглавлению](#intro)

<a id='3'></a> 

### Замена типа данных

In [24]:
data.info() # обратим внимание на типы данных в датафрейме

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [25]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,6799.624714,42,среднее,1,женат / замужем,0,F,госслужащий,0,290780.735919,покупка коммерческой недвижимости
2,1,265.648026,42,среднее,1,женат / замужем,0,F,сотрудник,1,281249.736191,строительство собственной недвижимости
3,1,2115.535249,42,среднее,1,женат / замужем,0,F,госслужащий,1,79147.663344,покупка жилья для сдачи
4,1,159.837829,42,начальное,3,женат / замужем,0,F,сотрудник,0,78870.858159,на покупку подержанного автомобиля


Для ежемесячного дохода мы не будем менять тип данных, но улучшим восприятие денежных сумм, округлив значения до 2 знаков после запятой:

In [26]:
data['total_income'] = data['total_income'].round(2)

А для трудового стажа, изменим тип данных на целочисленный, ведь стаж должен быть представлен в целых днях. Используем метод `astype` - чтобы перевести значения из `float` в нужный нам тип `int`.

In [27]:
data['days_employed'] = data['days_employed'].astype('int64')
print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB
None


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,6799,42,среднее,1,женат / замужем,0,F,госслужащий,0,290780.74,покупка коммерческой недвижимости
2,1,265,42,среднее,1,женат / замужем,0,F,сотрудник,1,281249.74,строительство собственной недвижимости
3,1,2115,42,среднее,1,женат / замужем,0,F,госслужащий,1,79147.66,покупка жилья для сдачи
4,1,159,42,начальное,3,женат / замужем,0,F,сотрудник,0,78870.86,на покупку подержанного автомобиля


**Вывод**

На этом этапе мы улучшили отображение дохода, в соответствии с денежным форматом и изменили тип данных в стаже работы, для его корректного отображения.

[Назад к оглавлению](#intro)

<a id='4'></a> 

### Обработка дубликатов

In [28]:
data.duplicated().sum() # узнаем количество дубликатов

72

Дубликаты могли возникнуть из-за того что клиент дважды обращался за получением кредита. С другой стороны возможно что это и не дубликаты, так как люди с одинаковыми данными могут встречаться.

In [29]:
data = data.drop_duplicates().reset_index(drop=True) # удалим дубликаты
data.duplicated().sum()

0

**Вывод**

Всего было 72 дубликата от которых мы успешно избавились.

[Назад к оглавлению](#intro)

<a id='5'></a> 

### Лемматизация

Лемматизация поможет определить основные цели на которые клиенты берут кредит.

In [30]:
from pymystem3 import Mystem # импортируем нужные библиотеки
m = Mystem()
from collections import Counter

text = ' '.join(data['purpose'].unique()) # формируем текст только из уникальных целей
lemmas = m.lemmatize(text) # лемматизируем
print(Counter(lemmas)) # подсчитываем самые популярные цели

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'на': 4, 'свой': 4, 'операция': 4, 'строительство': 3, 'получение': 3, 'высокий': 3, 'свадьба': 3, 'коммерческий': 2, 'для': 2, 'подержать': 2, 'жилой': 2, 'заниматься': 2, 'дополнительный': 2, 'сделка': 2, 'собственный': 1, 'сдача': 1, 'ремонт': 1, 'приобретение': 1, 'семья': 1, 'со': 1, 'профильный': 1, 'проведение': 1, 'сыграть': 1, '\n': 1})


Выбираем самые часто встречающиеся существительные, это и будут наши категории для кредитов:
* недвижимость(жилье)
* автомобиль
* образование
* свадьба

In [31]:
def lemmas_purpose(row): # напишем функцию для емкого описания цели кредита
    purpose = row['purpose'] # добавим переменную с целью кредита для каждого значения в столбце
    purpose_lemmas = m.lemmatize(purpose)
 
    if ('недвижимость' in purpose_lemmas) or ('жилье' in purpose_lemmas): # поскольку под жильем и недвижимостью подразумеваются схожие понятия, то объединим их
        return 'недвижимость'
    elif 'автомобиль'in purpose_lemmas:
        return 'автомобиль'
    elif 'образование' in purpose_lemmas:
        return 'образование'
    elif 'свадьба' in purpose_lemmas:
        return 'свадьба'
    else:
        return 'цель не определена' # если вдруг мы не определили какую то цель, то после выполнения функции сможем детально просмотреть эти результаты

data['purpose_mod'] = data.apply(lemmas_purpose, axis=1) # методом apply применим написанную нами функцию к датафрейму и сохраним результат в новый столбец   
data['purpose_mod'].value_counts() 

недвижимость    10810
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose_mod, dtype: int64

**Вывод**

Как видно самой востребованной целью кредита являются операции с недвижимостью, затем люди хотят купить машину или получить образование и меньше всего людей берут кредит для проведения свадьбы.

[Назад к оглавлению](#intro)

<a id='6'></a> 

### Категоризация данных

Методом `.drop_duplicates()` мы можем проанализировать на какие категории поделить клиентов банка.

In [32]:
data[['education', 'education_id']].drop_duplicates() # по образованию

,education,education_id
0,высшее,0
1,среднее,1
4,начальное,3
63,неоконченное высшее,2
540,ученая степень,4


In [33]:
data[['children']].drop_duplicates() # по количеству детей

,children
0,1
79,0
186,3
238,2
733,4
740,5


**Вывод**

В данных представленны различные категории по которым можно разделить клиентов, благодаря этому можно найти целевую аудиторию для того или иного продукта банка.

[Назад к оглавлению](#intro)

<a id='7'></a>

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Создадим сводную таблицу методом `pivot_table()`. Аргументы метода:
* index — столбец или столбцы, по которым группируют данные (количество детей);
* values — значения, по которым мы хотим увидеть сводную таблицу (задолженность по кредиту);
* aggfunc — функция, применяемая к значениям (количество должников, всего клиентов в конкретной категории, процент должников от числа всех клиентов взявших кредит в этой категории)

In [34]:
first_report = data.pivot_table(index=['children'], values='debt', aggfunc=['sum', 'count', 'mean']) # создаем сводную таблицу
first_report.columns = ['debtors', 'total', '%'] # переименуем столбцы
first_report['%'] = (first_report['%'] * 100).round(1) # оформим визуализацию процента должников
first_report = first_report.sort_values(by=['total'], ascending=False) # отсортируем данные по количеству клиентов в категории
first_report # выведем результат

,debtors,total,%
children,,,
0,1063,14090,7.5
1,445,4855,9.2
2,202,2128,9.5
3,27,330,8.2
4,4,41,9.8
5,0,9,0.0


**Вывод**

На основании сводной таблицы, можно сделать вывод о том, что количество детей влияет на возврат кредита в срок следующим образом:
* **без детей** - это самая большая категория клиентов, в которой почти наименьший процент должников. Выдача кредитов этой категории клиентов, для банка менее рискована;
* **от 1 до 4 детей** - для этих категорий процент должников стремится к 10%, что не так выгодно для банка;
* **5 детей** - на удивление здесь вообще нет должников, возможно клиенты этой категории имеют какие то льготы позволяющие им лучше планировать свой бюджет.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
second_report = data.pivot_table(index=['family_status', 'family_status_id'], values='debt', aggfunc=['sum', 'count', 'mean'])
second_report.columns = ['debtors', 'total', '%']
second_report['%'] = (second_report['%'] * 100).round(1)
second_report = second_report.sort_values(by=['total'], ascending=False)
second_report

,,debtors,total,%
family_status,family_status_id,,,
женат / замужем,0,931,12338,7.5
гражданский брак,1,388,4151,9.3
не женат / не замужем,4,274,2810,9.8
в разводе,3,85,1195,7.1
вдовец / вдова,2,63,959,6.6


**Вывод**

На основании сводной таблицы можно сделать вывод о том, что:
+ Люди в официально зарегистрированном браке более ответственно относятся к планированию бюджета семьи, нежели клиенты в гражданском браке, возможно, это связано с юридической ответственностью между супругами.
+ Те кто не женат/ не замужем являются клиентами с самым высоким процентом должников.
+ Люди находящиеся в разводе или потерявшие супруга, наименее рискованные клиенты для банка.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на этот вопрос добавим столбец с категориями дохода клиентов.

In [36]:
def month_income(row): # напишем функцию
    if row <= 50000:
        return 'доход до 50к'
    elif row <= 100000:
        return 'доход до 100к '
    elif row <= 150000:
        return 'доход до 150к'
    elif row <= 200000:
        return 'доход до 200к'
    else:
        return 'доход свыше 200к'
    

data['income_category'] = data['total_income'].apply(month_income) # применим метод apply к столбцу с ежемесячными доходами

In [37]:
third_report = data.pivot_table(index=['income_category'], values='debt', aggfunc=['sum', 'count', 'mean'])
third_report.columns = ['debtors', 'total', '%']
third_report['%'] = (third_report['%'] * 100).round(1)
third_report = third_report.sort_values(by=['total'], ascending=False)
third_report

,debtors,total,%
income_category,,,
доход до 150к,624,7159,8.7
доход свыше 200к,358,5067,7.1
доход до 200к,405,4764,8.5
доход до 100к,331,4091,8.1
доход до 50к,23,372,6.2


**Вывод**

На основании данных из сводной таблицы видно:
+ **доход до 50к и более 200к** - клиенты с такой вилкой дохода ответственней подходят к оплате кредита
+ **доход от 50к и до 200к** - в этих категориях для банка выше риск просрочки по кредитам

- Как разные цели кредита влияют на его возврат в срок?

In [38]:
four_report = data.pivot_table(index=['purpose_mod'], values='debt', aggfunc=['sum', 'count', 'mean'])
four_report.columns = ['debtors', 'total', '%']
four_report['%'] = (four_report['%'] * 100).round(1)
four_report = four_report.sort_values(by=['total'], ascending=False)
four_report

,debtors,total,%
purpose_mod,,,
недвижимость,782,10810,7.2
автомобиль,403,4306,9.4
образование,370,4013,9.2
свадьба,186,2324,8.0


**Вывод**

По результатам этой таблицы:
+ **недвижимость** - цель с наименьшим риском на исправный возврат по кредиту;
+ **автомобиль** - самай рикованная цель кредита, возможная причина, что это имущество может привести к дополнительным тратам, к которым клиент не будет готов;
+ **образование** - почти такие же риски как и с покупкой авто;
+ **свадьба** - средние риски по возврату кредита.

[Назад к оглавлению](#intro)

<a id='8'></a>

## Шаг 4. Общий вывод

Мы проверили четыре гипотезы, на основании результатов которых были выделены более предпочтительные группы клиентов и категории целей кредитования. Учитывая полученные данные банк может принять решение о повышении кредитного лимита по лучшим категориям. Так, например, покупка недвижимости для состоящих в браке клиентов, либо понижение одобренной суммы кредитования для желающих купить автомобиль с доходом до 100к.
Это исследование поможет банку снизить финансовые риски и общее количество задолжностей по кредитам. А также в будущем разработать продукты удовлетворяющие потребностям целевой аудитории, способной оплачивать кредит в срок, тем самым увеличив доход банка.

[Назад к оглавлению](#intro)